In [33]:
from ShapeVVE.DataLoader.fetcher import DataFetcher
import torch
from ShapeVVE.DataLoader.loadDataFromAfrr import get_UEA_dataset

# 1.加载数据集
DATASET_PATH = "../../../../data_files/Multivariate2018_arff/Multivariate_arff"
x_train, y_train, x_val, y_val, x_test, y_test=get_UEA_dataset(DATASET_PATH,"BasicMotions")
print(f"x_train: {x_train.shape}, y_train: {y_train.shape}, x_val: {x_val.shape}, y_val: {y_val.shape}, x_test: {x_test.shape}, y_test: {y_test.shape}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_train = torch.from_numpy(x_train).to(device)
x_val = torch.from_numpy(x_val).to(device)
x_test = torch.from_numpy(x_test).to(device)
y_train = torch.from_numpy(y_train).to(device)
y_val = torch.from_numpy(y_val).to(device)
y_test = torch.from_numpy(y_test).to(device)

n_classes = len(torch.unique(y_train))
in_channels=len(x_train[0])

datafetcher=DataFetcher.from_data_splits(
        x_train=x_train,
        y_train=y_train,
        x_valid=x_val,
        y_valid=y_val,
        x_test=x_test,
        y_test=y_test,
        one_hot=False,
    )

print(f'n_classes: {n_classes}')

Shapes - Train: (40, 6, 100), Val: (8, 6, 100), Test: (32, 6, 100)
x_train: (40, 6, 100), y_train: (40,), x_val: (8, 6, 100), y_val: (8,), x_test: (32, 6, 100), y_test: (32,)
n_classes: 4


In [34]:
from torch.optim import Adam
from torch import nn
import torch
from ShapeVVE.ShapeletsVE.shapeletsVE import DVRLChannel
from ShapeVVE.Evaluate.api import ExperimentMediatorChannel
from ShapeVVE.PredictorModel.LearningShapeletsDimension import LearningShapeletsDimensionSelection

metric_name="accuracy"
shapelets_size_and_len = {15: 10,  30: 10}

train_kwargs = {"epochs": 50, "batch_size": 100, "lr": 0.01}
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model = LearningShapeletsDimensionSelection(
    shapelets_size_and_len=shapelets_size_and_len,  # 增加shapelet数量
    loss_func=nn.CrossEntropyLoss(),
    in_channels=in_channels,  # 确保与数据实际通道数一致
    num_classes=n_classes,
    dist_measure='euclidean',
    verbose=1,
    to_cuda=torch.cuda.is_available(),
    k=5,           # 增大最近邻数量
    l1=0.01,        # 降低距离损失权重
    l2=0.001,        # 降低相似性损失权重
    lr=0.01,
    weight_decay=1e-4
)

exper_med = ExperimentMediatorChannel(
    fetcher=datafetcher,
    pred_model=model,
    train_kwargs=train_kwargs,
    metric_name=metric_name,
)

data_evaluators = [DVRLChannel(rl_epochs=2000,rl_batch_size=10000,device=device)]

# compute data values.
exper_med = exper_med.compute_data_values(shapelets_num_sum=20,data_evaluators=data_evaluators)
print([attr for attr in dir(exper_med) if not callable(getattr(exper_med, attr)) and not attr.startswith('__')])

result=exper_med.evaluate_shapeletVE_model(data_evaluators=data_evaluators)
# 根据传入的函数，存储精度等评估结果,评估精度

print(f'execute end')

metric_name is Metrics(metric_name)



  0%|          | 0/50 [00:00<?, ?it/s]
Loss CE: 5.118203639984131, Loss dist: 0.04628797993063927, Loss sim: 0.00020897509239148349:   0%|          | 0/50 [00:00<?, ?it/s]
Loss CE: 5.118203639984131, Loss dist: 0.04628797993063927, Loss sim: 0.00020897509239148349:   2%|▏         | 1/50 [00:00<00:07,  6.80it/s]
Loss CE: 2.725332260131836, Loss dist: 0.046140268445014954, Loss sim: 0.00020890591258648783:   2%|▏         | 1/50 [00:00<00:07,  6.80it/s]
Loss CE: 1.5792109966278076, Loss dist: 0.04601456597447395, Loss sim: 0.00020877239876426756:   2%|▏         | 1/50 [00:00<00:07,  6.80it/s]
Loss CE: 1.5792109966278076, Loss dist: 0.04601456597447395, Loss sim: 0.00020877239876426756:   6%|▌         | 3/50 [00:00<00:03, 13.00it/s]
Loss CE: 1.5443495512008667, Loss dist: 0.04591156542301178, Loss sim: 0.00020866347767878324:   6%|▌         | 3/50 [00:00<00:03, 13.00it/s]
Loss CE: 1.8533817529678345, Loss dist: 0.04583284258842468, Loss sim: 0.00020870211301371455:   6%|▌         | 3/50 [

bathch number: 10000 1



  0%|          | 0/1 [00:00<?, ?it/s]

train batch : <built-in method size of Tensor object at 0x16edc5720> <built-in method size of Tensor object at 0x16ee1e450> <built-in method size of Tensor object at 0x16ee1e3b0>
the 1 -th batch




  0%|          | 0/50 [00:00<?, ?it/s]

Loss CE: 1.791213035583496, Loss dist: 0.018731627613306046, Loss sim: 9.008294728118926e-05:   0%|          | 0/50 [00:00<?, ?it/s]

Loss CE: 1.351381778717041, Loss dist: 0.018699944019317627, Loss sim: 9.011603106046095e-05:   0%|          | 0/50 [00:00<?, ?it/s]

Loss CE: 1.3734897375106812, Loss dist: 0.018676992505788803, Loss sim: 9.014709939947352e-05:   0%|          | 0/50 [00:00<?, ?it/s]

Loss CE: 1.3734897375106812, Loss dist: 0.018676992505788803, Loss sim: 9.014709939947352e-05:   6%|▌         | 3/50 [00:00<00:02, 23.16it/s]

Loss CE: 1.4052700996398926, Loss dist: 0.018655993044376373, Loss sim: 9.016234253067523e-05:   6%|▌         | 3/50 [00:00<00:02, 23.16it/s]

Loss CE: 1.3698861598968506, Loss dist: 0.018635492771863937, Loss sim: 9.016710100695491e-05:   6%|▌         | 3/50 [00:00<00:02, 23.16it/s]

Loss CE: 1.2940438985824585, Loss dist: 0.0186165738850832, Loss sim: 9.01659150258638e-05:   6%|▌         | 3/50 [00:00<00:02

----  log_losses  ----
50
dim_weights:torch.Size([40, 6]),selected_mask:torch.Size([20, 6]),reward:-0.75



  0%|          | 0/50 [00:00<?, ?it/s]
Loss CE: 1.4533252716064453, Loss dist: 0.022898267954587936, Loss sim: 0.00011063890269724652:   0%|          | 0/50 [00:00<?, ?it/s]
Loss CE: 1.0146799087524414, Loss dist: 0.022855427116155624, Loss sim: 0.0001106472045648843:   0%|          | 0/50 [00:00<?, ?it/s] 
Loss CE: 1.0312906503677368, Loss dist: 0.0228279959410429, Loss sim: 0.00011066719889640808:   0%|          | 0/50 [00:00<?, ?it/s] 
Loss CE: 1.0312906503677368, Loss dist: 0.0228279959410429, Loss sim: 0.00011066719889640808:   6%|▌         | 3/50 [00:00<00:02, 23.25it/s]
Loss CE: 1.0912678241729736, Loss dist: 0.02280859835445881, Loss sim: 0.00011072235065512359:   6%|▌         | 3/50 [00:00<00:02, 23.25it/s]
Loss CE: 1.0622491836547852, Loss dist: 0.022794025018811226, Loss sim: 0.00011079943215008825:   6%|▌         | 3/50 [00:00<00:02, 23.25it/s]
Loss CE: 0.9822630882263184, Loss dist: 0.022782597690820694, Loss sim: 0.00011088942846981809:   6%|▌         | 3/50 [00:00<00:02

DVRLChannel(rl_epochs=2000, rl_batch_size=10000, device=cpu)
Elapsed time DVRLChannel(rl_epochs=2000, rl_batch_size=10000, device=cpu): 0:00:07.437092
['data_evaluators', 'fetcher', 'num_data_eval', 'raise_error', 'timings', 'train_kwargs']
{'accuracy': 1.0}
execute end
